In [ ]:
import glob
import os
import numpy as np
import pandas as pd
import random
import itertools
import matplotlib.pyplot as plt

from healthcareai_toolkit.clients import MedImageInsightClient
from healthcareai_toolkit import settings

In [ ]:
import logging

logging.basicConfig(level=logging.DEBUG)
logging.info("Just printing logging")

In [ ]:
N = 1

data_dir = settings.DATA_ROOT

files = list(glob.glob(data_dir + "**/*.*", recursive=True))
files = [f for f in files if os.path.isfile(f)]
files_dict = {
    k: list(v) for k, v in itertools.groupby(files, lambda x: x.split(".")[-1])
}

print(files_dict.keys())
files = sum([files_dict[k][:N] for k in ["jpg", "dcm", "png", "jpeg"]], [])

files

In [ ]:
client = MedImageInsightClient(settings.MI2_MODEL_ENDPOINT)

In [ ]:
images = files
np_img = client.read_to_image_array(
    os.path.join(
        data_dir,
        "medimageinsight-classification/images/1.3.6.1.4.1.55648.020128023405162813637635556625961837376.1002.1002.green.dcm",
    )
)
np_img2 = client.read_to_image_array(
    os.path.join(data_dir, "cxrreportgen-images/cxr_lateral.jpg")
)

images.append(np_img)
images.append(np_img2)

print(np_img.dtype, np_img2.dtype)

In [ ]:
result = client.submit(image_list=images)

In [ ]:
for v in result:
    print(v["image_features"][:5])

In [ ]:
import numpy as np


def display_vector_as_image(
    vector: np.ndarray,
    display_width: int = 100,
    colormap: str = "bwr",
    vmin=-0.25,
    vmax=0.25,
):
    N = len(vector)
    image = np.tile(vector.reshape((1, N)), (display_width, 1))
    plt.imshow(image, cmap=colormap, vmin=vmin, vmax=vmax)
    plt.axis("off")
    plt.show()


vector = np.array(result[0]["image_features"])
display_vector_as_image(vector.flatten())